In [154]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm 
from tqdm import trange
import time
from google_play_scraper import app, Sort, reviews, reviews_all
from pprint import pprint
import json
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re

In [42]:
url = 'https://play.google.com/store/apps/details?id=com.clarocolombia.miclaro&hl=es_CO&gl=US&showAllReviews=true'

# Obtencio de datos mediante Libreria <strong>google_play_scraper</strong>

### Consultar los datos generales de la app

In [43]:
result = app(
    'com.clarocolombia.miclaro',
    lang='es',
    country='us' 
)

### Consulta de comentarios por lotes

In [44]:
rvws, token = reviews(
        'com.clarocolombia.miclaro', 
        lang='es',            
        country='us',         
        sort=Sort.NEWEST,     
        filter_score_with=None,  
        count=100             
    )

In [59]:
rvws[0:2]

[{'reviewId': 'gp:AOqpTOFjz5e9ioJtbi-HvBz7SH9XQ5_IoGk9U8Pr5s02BIu0Jrcgp1r3oPxYYDt95EowiySUpb9C_hla1VqvAdU',
  'userName': 'Carlos Ortega',
  'userImage': 'https://play-lh.googleusercontent.com/a/AATXAJzsdSEnAgfmcUNHAxe7LTsGKuXHgDG6274Z5Ovn=mo',
  'content': 'Muy buena aplicacion',
  'score': 5,
  'thumbsUpCount': 0,
  'reviewCreatedVersion': '11.4.0',
  'at': datetime.datetime(2021, 7, 30, 21, 39, 6),
  'replyContent': None,
  'repliedAt': None},
 {'reviewId': 'gp:AOqpTOGPMRc3QrLlDSZ5XVM_pgQImI8P9Ofkgnp0ZjmcljJQJ1Iuobqglp-stQt-hG3_7ytsRZ-lV21iwDXDLcI',
  'userName': 'Thony Gutierrez',
  'userImage': 'https://play-lh.googleusercontent.com/a/AATXAJwdaKc1y-zE0P0tPyli--fESJtdzeDAI_Qu-iG5=mo',
  'content': 'Que servicio tan malo, intento comunicarme con ustedes y no hay forma, escribo por WhatsApp y responden cuando les da la gana, hay una línea nacional que no sirve, y el servicio de *611 demoran en recibir la llamada, pero para generar una bendita factura si de una están todos los canales

### Consulta de todos los cometarios

In [45]:
all_rvws = reviews_all(
    'com.clarocolombia.miclaro',
    sleep_milliseconds=0,
    lang='es', 
    country='us',
    sort=Sort.NEWEST, 
    filter_score_with=None
)

In [50]:
print ("La cantidad de comentarios obtenidos de la aplicacion fueron: "+ str(len(all_rvws)))

La cantidad de comentarios obtenidos de la aplicacion fueron: 86845


In [60]:
all_rvws[0:2]

[{'reviewId': 'gp:AOqpTOFjz5e9ioJtbi-HvBz7SH9XQ5_IoGk9U8Pr5s02BIu0Jrcgp1r3oPxYYDt95EowiySUpb9C_hla1VqvAdU',
  'userName': 'Carlos Ortega',
  'userImage': 'https://play-lh.googleusercontent.com/a/AATXAJzsdSEnAgfmcUNHAxe7LTsGKuXHgDG6274Z5Ovn=mo',
  'content': 'Muy buena aplicacion',
  'score': 5,
  'thumbsUpCount': 0,
  'reviewCreatedVersion': '11.4.0',
  'at': datetime.datetime(2021, 7, 30, 21, 39, 6),
  'replyContent': None,
  'repliedAt': None},
 {'reviewId': 'gp:AOqpTOGPMRc3QrLlDSZ5XVM_pgQImI8P9Ofkgnp0ZjmcljJQJ1Iuobqglp-stQt-hG3_7ytsRZ-lV21iwDXDLcI',
  'userName': 'Thony Gutierrez',
  'userImage': 'https://play-lh.googleusercontent.com/a/AATXAJwdaKc1y-zE0P0tPyli--fESJtdzeDAI_Qu-iG5=mo',
  'content': 'Que servicio tan malo, intento comunicarme con ustedes y no hay forma, escribo por WhatsApp y responden cuando les da la gana, hay una línea nacional que no sirve, y el servicio de *611 demoran en recibir la llamada, pero para generar una bendita factura si de una están todos los canales

## Conversion del JSON obtenido a un Dataframe para mas comodidad en el procesamiento de datos

In [57]:
df = pd.json_normalize(all_rvws)

In [61]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOFjz5e9ioJtbi-HvBz7SH9XQ5_IoGk9U8Pr5s0...,Carlos Ortega,https://play-lh.googleusercontent.com/a/AATXAJ...,Muy buena aplicacion,5,0,11.4.0,2021-07-30 21:39:06,None,NaT
1,gp:AOqpTOGPMRc3QrLlDSZ5XVM_pgQImI8P9Ofkgnp0Zjm...,Thony Gutierrez,https://play-lh.googleusercontent.com/a/AATXAJ...,"Que servicio tan malo, intento comunicarme con...",1,0,11.4.0,2021-07-30 21:17:47,None,NaT
2,gp:AOqpTOGjeH1K5sGpZlPj_2fkqRx6m6s0_jD4wLscjux...,Alfonso Gomez,https://play-lh.googleusercontent.com/a/AATXAJ...,Excelente Servicio muy bien muchas gracias,5,0,11.4.0,2021-07-30 20:52:06,None,NaT
3,gp:AOqpTOE4Wm3ydshHQrh0f7XDu9R7gzidN4Z6BVrTngB...,gustavo toloza,https://play-lh.googleusercontent.com/a-/AOh14...,Desde la actualización realizada el 28-07-2021...,1,0,11.4.0,2021-07-30 20:29:43,None,NaT
4,gp:AOqpTOFQt_hg8N5dGhSEw2-0FwOtEINg6nB5VpqFhwV...,diego luis arrubla castrillon,https://play-lh.googleusercontent.com/a-/AOh14...,Si lleva más de una semana mala la señal no se...,1,0,8.7.2,2021-07-30 20:28:51,None,NaT


# Obtencio de datos mediante Selenium (Web scraping)

In [80]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\CRISTIAN\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


### Se simula el scrolldown infinitamente con selenium para seguir cargando mas comentarios

In [81]:
SCROLL_PAUSE_TIME = 1

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(SCROLL_PAUSE_TIME)

    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height
    
url_soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

### Se busca la seccion de comentarios en base a un titulo en el html

In [87]:
seccion_de_comentarios = url_soup.find("h3", text="Opiniones de usuarios")
seccion_de_comentarios

<h3 class="td1D0d">Opiniones de usuarios</h3>

### Se obtiene el div que contiene todos los comentarios

In [124]:
comentarios = seccion_de_comentarios.find_next_sibling("div")
print(str(comentarios)[0:1000])

<div jsname="fk8dgd"><div jscontroller="H6eOGe" jsdata="yf3HXc;gp:AOqpTOG0hIJl-Bs382MQOjLDMc6T861bvcPMHNvDnL8XAdXGJ-VvaiWpOaffzVi9BN1_SaaVT4oqaPEXDlnTAwM;$3 YjFXEf;_;$4" jsmodel="y8Aajc"><div class="zc7KVe"><div aria-hidden="true" class="vDSMeb bAhLNe"><img aria-hidden="true" class="T75of ZqMJr" data-iml="846.2000000011176" src="https://play-lh.googleusercontent.com/a/AATXAJza5PnPeu2mF3SeGGQCtXjzhtwXISPY7hpIsSSn=w48-h48-n-rw-mo" srcset="https://play-lh.googleusercontent.com/a/AATXAJza5PnPeu2mF3SeGGQCtXjzhtwXISPY7hpIsSSn=w96-h96-n-rw-mo 2x"/></div><div class="d15Mdf bAhLNe"><div class="xKpxId zc7KVe"><div class="bAhLNe kx8XBd"><span class="X43Kjb">Cecilia Rodríguez</span><div><span class="nt2C1d"><div class="pf5lIe"><div aria-label="Calificación: 1 de cinco estrellas" role="img"><div class="vQHuPe bUWb7c"></div><div class="L0jl5e bUWb7c"></div><div class="L0jl5e bUWb7c"></div><div class="L0jl5e bUWb7c"></div><div class="L0jl5e bUWb7c"></div></div></div></span><span class="p2TkOb">July 2

### Obtencion de comentarios

In [138]:
textos = comentarios.find_all('div', class_ = "UD7Dzf")

In [141]:
textos[0].text

' Claro es lo peor últimamente en telefonía celular, no hay señal para las llamadas y mucho menos para datos, he llamado varias veces sin solución, después de muchos años desde que era celcaribe, me tocó cambiar de operador. En telefonía fija el internet nunca funciona, ya estoy cansada de está situac…Ver opinión completaClaro es lo peor últimamente en telefonía celular, no hay señal para las llamadas y mucho menos para datos, he llamado varias veces sin solución, después de muchos años desde que era celcaribe, me tocó cambiar de operador. En telefonía fija el internet nunca funciona, ya estoy cansada de está situación, uno llama y cuando vienen no le SOLUCIONAN Nada.'

In [143]:
len(textos)

80845

### Obtencion de estrellas

In [146]:
estrellas = comentarios.find_all('div', class_ = "pf5lIe")

In [148]:
estrellas[0]

<div class="pf5lIe"><div aria-label="Calificación: 1 de cinco estrellas" role="img"><div class="vQHuPe bUWb7c"></div><div class="L0jl5e bUWb7c"></div><div class="L0jl5e bUWb7c"></div><div class="L0jl5e bUWb7c"></div><div class="L0jl5e bUWb7c"></div></div></div>

In [158]:
calificacion = re.search("(?<=Calificación: )(.*)(?= de)", str(estrellas[0])).group(1)
calificacion

'1'

In [160]:
len(estrellas)

80845

### Agrupamiento de los datos deseados 

In [164]:
lista_comentarios = []
lista_calificaciones = []

for i in range(len(textos)):
    lista_comentarios.append(textos[i].text)
    lista_calificaciones.append(re.search("(?<=Calificación: )(.*)(?= de)", str(estrellas[i])).group(1))

In [167]:
lista_comentarios[0:3]

[' Claro es lo peor últimamente en telefonía celular, no hay señal para las llamadas y mucho menos para datos, he llamado varias veces sin solución, después de muchos años desde que era celcaribe, me tocó cambiar de operador. En telefonía fija el internet nunca funciona, ya estoy cansada de está situac…Ver opinión completaClaro es lo peor últimamente en telefonía celular, no hay señal para las llamadas y mucho menos para datos, he llamado varias veces sin solución, después de muchos años desde que era celcaribe, me tocó cambiar de operador. En telefonía fija el internet nunca funciona, ya estoy cansada de está situación, uno llama y cuando vienen no le SOLUCIONAN Nada.',
 ' Tiene buenas opciones en la app como tal. Lástima que no funcionen. Se supone que al ser una empresa tan grande deberían tener una buena app y más aún en tiempos de pandemia, cuando para muchas personas es imposible o "no seguro" salir a hacer trámites de pagos y otros tantos. Pésima app e inútil di…Ver opinión comp

In [168]:
lista_calificaciones[0:3]

['1', '1', '2']

### Creacion de un dataframe con los datos obtenidos del web scrapping

In [169]:
df_scrapping = pd.DataFrame(columns=['Comentarios','Calificaciones'])
df_scrapping['Comentarios'] = lista_comentarios
df_scrapping['Calificaciones'] = lista_calificaciones
df_scrapping.head()

,Comentarios,Calificaciones
0,Claro es lo peor últimamente en telefonía cel...,1
1,Tiene buenas opciones en la app como tal. Lás...,1
2,"Desde hace un año que compré éste paquete, ha...",2
3,Llevo más de un año con el servicio de claro ...,3
4,La App de Claro es buena. Lo que no es bueno ...,4


# Conclusiones
Al comparar los dos metodos tanto en su tiempo de ejecucion como en la cantidad de comentarios obtenidos el ganador aparente fue la libreria <strong>google_play_scraper</strong> respecto al webscrapping empleando selenium, esto se debe a que el tiempo de obtencion de datos mediante la libreria fue de aproximadamente 2 minutos contra los 15 minutos que tomo selenium en hacer el webscrapping, ademas que el uso de la libreria es mucho mas sencillo y flexible respecto al webscrapping, por lo que esta queda seleccionada para su uso en la API.

### Se guardan los datos para su uso posterior

In [176]:
file_name = "../DATA/reviews_app_claro.csv"
df.to_csv(file_name, encoding='utf-8', index= False)